# Challenge 2: Subject Invariant Representation

## Predicting psychopathological trait from EEG and behavioral data

Finding biomarkers for psychopathology has always been a golden goal. As behavioral symptoms can be confounding between different pathologies the hope is that physiology signals give a more precise way of stratifying patients. Furthermore, collecting clinical data is costly while there are more abundant data resource that are non-clinincal and available. In this challenge, you will explore how to leverage large data recording to predict clinical factor in a non-supervised manner. We hope that you can advance clinical diagnosis for adolescent!

In this start kit, we will show you how to obtain the data for the challenge as well as preprocess and format it into a form processable by deep learning using EEGDash and Braindecode. We will then show an example of using Self-Supervised Learning to train the model on the data without labels, and fine-tune it for clinical prediction at inference time. This is by no means a requirement of the competition. The goal is to give you an jumping-off point and some inspiration. We will provide some suggestions of areas to explore at the end of the start kit, but feel free to be creative and choose whichever approach you see fit for the problem!


## Summary table for this start kit

**Contents**:

1. Loading data
2. Preprocessing
2. Preparing the dataloaders.
3. Building the deep learning model.
4. Designing SSL task and the training loop.
5. Training SSL model.
6. Designing regression model for p-factor prediction.
7. Going further

| The tutorial assumes some prior knowledge of deep learning concepts and EEG concepts:|
| --- |
|* Standard neural network architectures, e.g., convolutional neural networks|
|* Optimization by batch gradient descent and backpropagation|
|* Overfitting, early stopping, regularisation |
|* Some knowledge of pytorch and, optionally, of the pytorch Lightning framework|
|* That you know what EEG is |
|* that you have basic familiarity with EEG preprocessing |
|* Like and support open-source :) |

*NOTE: You will still be able to run the whole notebook at your own pace and learn about these concepts along the way*

*We will provide some complementary references if you need to familiarise yourself with the necessary concepts.*

In [ ]:
!pip install jupyter
!pip install git+https://github.com/sccn/eegdash.git
!pip install git+https://github.com/braindecode/braindecode.git
!pip install lightning

In [1]:
from pathlib import Path

from eegdash import EEGChallengeDataset
from braindecode.datasets.base import EEGWindowsDataset, BaseConcatDataset, BaseDataset
import torch
import random
from torch.utils.data import DataLoader
from torch import optim
from torch.nn.functional import l1_loss
from braindecode.models import EEGNetv4
import lightning as pl

### Getting the data

We will load the data using EEGDash and get a Braindecode Dataset object, which is a wrapper around PyTorch Dataset. We are showing how to load the data releaes 1 that has 136 subjects and only the Resting State task to reduce memory usage. In practice you can use all available releases and all tasks by updating the release_list and task_list.

In [2]:
# Set some paths to locate the data and cache/save results
DATA_DIR = Path("data")
DATA_DIR.mkdir(parents=True, exist_ok=True)

In [3]:
release_list = ["R1"] # Add more releases as needed, e.g., ["R1", "R2", "R3", "R4", "R5", "R6"]
task_list = ['RestingState'] # Add more tasks as needed, e.g., ['RestingState', 'ThePresent', 'surroundSupp']

all_datasets_list = [
    EEGChallengeDataset(
        release=release,
        query={'task': {'$in': task_list}},
        description_fields=[
            "subject",
            "session",
            "ntimes",
            "run",
            "task",
            "age",
            "gender",
            "sex",
            "p_factor",
        ],
        cache_dir=DATA_DIR,
    )
    for release in release_list
]
# Concatenate all datasets into a single BaseConcatDataset. 
# This is especially necessary when using multiple releases or tasks are used.
all_datasets = BaseConcatDataset(all_datasets_list)

print("Example recording description:")
all_datasets.datasets[0].description


Example recording description:


subject     NDARAC904DMU
ntimes           403.474
task        RestingState
age              11.3386
sex                    F
p_factor          -0.603
dtype: object

In [4]:
# Filter out recordings that are too short
all_datasets = BaseConcatDataset(
    [ds for ds in all_datasets.datasets if ds.description.ntimes >= 30]
)

print(f'Number of recordings valid recordings: {len(all_datasets.datasets)}')

Number of recordings valid recordings: 136


### Data preprocessing using Braindecode

In [5]:
from braindecode.preprocessing.preprocess import Preprocessor, preprocess

high_cut_hz = 30

preprocessors = [
    Preprocessor("filter", l_freq=None, h_freq=high_cut_hz, n_jobs=-1),
]

# Transform the data
preprocess(all_datasets, preprocessors)

Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 45 samples (0.450 s)



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done  98 out of 129 | elapsed:    0.7s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 124 out of 129 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 129 out of 129 | elapsed:    0.8s finished


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 45 samples (0.450 s)

Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 45 samples (0.450 s)



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  98 out of 129 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 124 out of 129 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 129 out of 129 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  98 out of 129 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 124 out of 129 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 129 out of 129 | elapsed:    0.1s finished


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 45 samples (0.450 s)

Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 45 samples (0.450 s)



...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 124 out of 129 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 129 out of 129 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  98 out of 129 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 124 out of 129 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 129 out of 129 | elapsed:    0.1s finished


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 45 samples (0.450 s)

Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 45 samples (0.450 s)



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 124 out of 129 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 129 out of 129 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    0.0s


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 45 samples (0.450 s)



[Parallel(n_jobs=-1)]: Done  98 out of 129 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 124 out of 129 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 129 out of 129 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  98 out of 129 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 124 out of 129 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 129 out of 129 | elapsed:    0.1s finished


### Create windows

In [6]:
from braindecode.preprocessing import create_fixed_length_windows
from sklearn.preprocessing import scale as standard_scale

SFREQ = 100
window_size_s = 30
window_size_samples = window_size_s * SFREQ
# Create 4-seconds windows with 2-seconds stride
windows_ds = create_fixed_length_windows(
    all_datasets,
    window_size_samples=window_size_samples,
    window_stride_samples=window_size_samples,
    drop_last_window=True,
)

# We also preprocess the windows by applying channel-wise z-score normalization.
preprocess(windows_ds, [Preprocessor(standard_scale, channel_wise=True)])

/home/dung/Documents/start-kit/.venv/lib/python3.10/site-packages/braindecode/preprocessing/preprocess.py:255: UserWarning: Applying preprocessors [<braindecode.preprocessing.preprocess.Preprocessor object at 0x7fab48866b90>] to the mne.io.Raw of an EEGWindowsDataset.
  warn(
/home/dung/Documents/start-kit/.venv/lib/python3.10/site-packages/braindecode/preprocessing/preprocess.py:255: UserWarning: Applying preprocessors [<braindecode.preprocessing.preprocess.Preprocessor object at 0x7fab48866b90>] to the mne.io.Raw of an EEGWindowsDataset.
  warn(
/home/dung/Documents/start-kit/.venv/lib/python3.10/site-packages/braindecode/preprocessing/preprocess.py:255: UserWarning: Applying preprocessors [<braindecode.preprocessing.preprocess.Preprocessor object at 0x7fab48866b90>] to the mne.io.Raw of an EEGWindowsDataset.
  warn(
/home/dung/Documents/start-kit/.venv/lib/python3.10/site-packages/braindecode/preprocessing/preprocess.py:255: UserWarning: Applying preprocessors [<braindecode.preproce

### Set up Self-Supervised Learning task

Self-Supervised Learning (SSL) has been shown to be an effective training paradigm to learn robust representation from the data without label. There are many different SSL "pre-text" tasks that one can use. In fact, figuring out which pre-text task is most effective for the data and the downstream tasks of interest is an important open research area. Here we will implement a pre-text task called Relative Positioning (RP) that has been applied to learn clinically meaningful representation from EEG data [Banville et al., (2020)](https://arxiv.org/abs/2007.16104). RP is a contrastive learning method where we randomly sample pairs of positive and negative EEG windows and train the model to predict this relationship in the embedding space. Braindecode already has a sampler for RP which we will use. For more detailed explanation of the task, check out the linked paper as well as this [tutorial](https://braindecode.org/stable/auto_examples/advanced_training/plot_relative_positioning.html) on braindecode.org

Here we train the RP task on the entire dataset without ever seeing the p-factor label. Later on, we will "fine-tune" the embedding model trained with SSL on labeled data for our p-factor prediction.

In [27]:
class RelativePositioningDataset(BaseConcatDataset):
    """BaseConcatDataset with __getitem__ that expects 2 indices and a target."""

    def __init__(self, list_of_ds):
        super().__init__(list_of_ds)
        self.return_pair = True

    def __getitem__(self, index):
        if self.return_pair:
            ind1, ind2, y = index
            return (super().__getitem__(ind1)[0], super().__getitem__(ind2)[0]), y
        else:
            return super().__getitem__(index)

    @property
    def return_pair(self):
        return self._return_pair

    @return_pair.setter
    def return_pair(self, value):
        self._return_pair = value

ssl_windows_ds = RelativePositioningDataset(windows_ds.datasets)

In [28]:
from braindecode.samplers import RelativePositioningSampler

tau_pos, tau_neg = int(SFREQ * 30), int(SFREQ * 30)
n_examples_train = 250 * len(ssl_windows_ds.datasets)
random_state = 42

ssl_sampler = RelativePositioningSampler(
    ssl_windows_ds.get_metadata(),
    tau_pos=tau_pos,
    tau_neg=tau_neg,
    n_examples=n_examples_train,
    same_rec_neg=False,
    random_state=random_state,
)

### Create embedding model

We will use EEGNet as an example

In [33]:
# Print model summary
from torchinfo import summary

pl.seed_everything(42)
emb_size = 100

model = EEGNetv4(n_chans=129, n_outputs=emb_size, n_times=30 * SFREQ)

# Create PyTorch Dataloader
train_dataloader = DataLoader(ssl_windows_ds, sampler=ssl_sampler, batch_size=128)
batch = next(iter(train_dataloader))
summary(model, input_data=batch[0][0])

Seed set to 42


Layer (type:depth-idx)                        Output Shape              Param #
EEGNetv4                                      [128, 100]                --
├─Ensure4d: 1-1                               [128, 129, 3000, 1]       --
├─Rearrange: 1-2                              [128, 1, 129, 3000]       --
├─Conv2d: 1-3                                 [128, 8, 129, 3001]       512
├─BatchNorm2d: 1-4                            [128, 8, 129, 3001]       16
├─ParametrizedConv2dWithConstraint: 1-5       [128, 16, 1, 3001]        --
│    └─ModuleDict: 2-1                        --                        --
│    │    └─ParametrizationList: 3-1          [16, 1, 129, 1]           2,064
├─BatchNorm2d: 1-6                            [128, 16, 1, 3001]        32
├─ELU: 1-7                                    [128, 16, 1, 3001]        --
├─AvgPool2d: 1-8                              [128, 16, 1, 750]         --
├─Dropout: 1-9                                [128, 16, 1, 750]         --
├─Conv2d: 1-10  

### Set up model training using PyTorch Lightning

PyTorch Lightning is a PyTorch wrapper that simplifies many steps of deep model training and experimentation. It also has the bonus ability to adapt to different hardware automatically and make the scaling process to multiple GPUs and nodes more straightforward.

In [35]:
import torch.nn as nn

class ContrastiveNet(nn.Module):
    """Contrastive module with linear layer on top of siamese embedder.

    Parameters
    ----------
    emb : nn.Module
        Embedder architecture.
    emb_size : int
        Output size of the embedder.
    dropout : float
        Dropout rate applied to the linear layer of the contrastive module.
    """
    def __init__(self, emb_size, dropout=0.5):
        super().__init__()
        self.clf = nn.Sequential(nn.Dropout(dropout), nn.Linear(emb_size, 1))

    def forward(self, x1, x2):
        return self.clf(torch.abs(x1 - x2)).flatten()

class BrainModule(pl.LightningModule):
    def __init__(
        self,
        embed: nn.Module,
        contrastive_net: nn.Module,
        loss_fn: nn.Module,
        learning_rate: float,
        weight_decay: float,
    ) -> None:
        super().__init__()
        self.embed = embed
        self.contrastive_net = contrastive_net
        self.loss_fn = loss_fn
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay

    def configure_optimizers(self) -> nn.Module:
        return torch.optim.AdamW(
            self.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x1, x2 = x
        z1, z2 = self.embed(x1), self.embed(x2)
        return self.contrastive_net(z1, z2)

    def predict_step(
        self, batch, batch_idx: int, dataloader_idx: int = 0,
    ) -> torch.Tensor:
        return self.forward(batch[0])

    def _run_step(self, batch) -> torch.Tensor:
        x, y_true = batch
        y_pred = self.forward(x)
        return self.loss_fn(y_pred, y_true)

    def training_step(
        self, batch, batch_idx: int,
    ) -> torch.Tensor:
        loss = self._run_step(batch)
        self.log("train_loss", loss, on_epoch=True, on_step=True, prog_bar=True)
        return loss

In [36]:
import copy

module = BrainModule(
    embed=copy.deepcopy(model),
    contrastive_net=copy.deepcopy(ContrastiveNet(emb_size)),
    loss_fn=nn.BCEWithLogitsLoss(),
    learning_rate=5e-3,
    weight_decay=0.01,
)

We create a `Trainer`, which will handle the training loop. The maxmium number of training epochs (or "passes" through the training set) is set with `max_epochs`. The `accelerator` is set to "auto". Lightning will pick up whichever hardward accelerator is currently available in the environment. If cuda is detected, it will automatically run our training on GPU

In [ ]:
trainer = pl.Trainer(
    max_epochs=2, # train RP for 2 epochs
    accelerator="auto",
)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/dung/Documents/start-kit/.venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [ ]:
# Takes about 2 minute per epoch on a single GPU
trainer.fit(
    model=module,
    train_dataloaders=train_dataloader,
)

You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type              | Params | Mode 
--------------------------------------------------------------
0 | embed           | EEGNetv4          | 152 K  | train
1 | contrastive_net | ContrastiveNet    | 101    | train
2 | loss_fn         | BCEWithLogitsLoss | 0      | train
--------------------------------------------------------------
152 K     Trainable params
0         Non-trainable params
152 K     Total params
0.609     Total estimated model params size (MB)
29        Modules in train mode
0         Modules in eval mode


/home/dung/Documents/start-kit/.venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.


### Apply trained model on p-factor prediction

Now that we have our embedding model, we will fine-tune it for the task of p-factor prediction. We will extract p-factor from our metadata and use it as label for training. Here we keep the current release R1 dataset for convenience. In practice you should load another release for p-factor regression fine-tuning. As with a typical supervised learning approach, we will split our dataset into training, validation, and test set and use our validation set to decide early stopping point.

In [37]:
class DatasetWrapper(BaseDataset):
    def __init__(self, dataset: EEGWindowsDataset, seed=None):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        X, _, inds = self.dataset[index]

        p_factor = torch.tensor(self.dataset.description["p_factor"])

        return X, p_factor, inds

In [38]:
import numpy as np
from sklearn.model_selection import train_test_split

random_state = 42  # Set a random state for reproducibility
subjects = np.unique(windows_ds.description["subject"])
subj_train, subj_test = train_test_split(
    subjects, test_size=0.4, random_state=random_state
)
subj_valid, subj_test = train_test_split(
    subj_test, test_size=0.5, random_state=random_state
)

In [39]:
split_ids = {"train": subj_train, "valid": subj_valid, "test": subj_test}
splitted = dict()
for name, values in split_ids.items():
    splitted[name] = BaseConcatDataset(
        [DatasetWrapper(ds) for ds in windows_ds.datasets if ds.description["subject"] in values]
    )

In [40]:
train_dataloader = DataLoader(splitted["train"], batch_size=10, shuffle=True)
valid_dataloader = DataLoader(splitted["valid"], batch_size=10, shuffle=False)
test_dataloader = DataLoader(splitted["test"], batch_size=10, shuffle=False)

In [45]:
class RegressorModule(pl.LightningModule):
    def __init__(
        self,
        embed: nn.Module,
        embed_size: int,
        loss_fn: nn.Module,
        learning_rate: float,
    ) -> None:
        super().__init__()
        self.embed = embed
        self.embed.eval()  # Ensure the embedder is in eval mode
        self.clf = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(embed_size, 1)
        )
        self.loss_fn = loss_fn
        self.learning_rate = learning_rate

    def configure_optimizers(self) -> nn.Module:
        return torch.optim.Adam(
            self.clf.parameters(), lr=self.learning_rate
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        with torch.no_grad():
            z = self.embed(x)
        return self.clf(z).flatten()

    def predict_step(
        self, batch, batch_idx: int, dataloader_idx: int = 0,
    ) -> torch.Tensor:
        return self.forward(batch[0])

    def _run_step(self, batch) -> torch.Tensor:
        x, y_true, _ = batch
        # y_true (p-factor values) might contain NaN values, filter them out and use the same indices for x
        mask = ~torch.isnan(y_true)
        y_true = y_true[mask]
        x = x[mask]
        y_pred = self.forward(x)

        return self.loss_fn(y_pred, y_true)

    def training_step(
        self, batch, batch_idx: int,
    ) -> torch.Tensor:
        loss = self._run_step(batch)
        self.log("train_loss", loss, on_epoch=True, on_step=True, prog_bar=True)
        return loss

    def validation_step(self, batch: tuple[torch.Tensor, torch.Tensor], batch_idx: int) -> None:
        loss = self._run_step(batch)
        self.log("valid_loss", loss, on_epoch=True, on_step=False, prog_bar=True)

    def test_step(self, batch: tuple[torch.Tensor, torch.Tensor], batch_idx: int) -> None:
        loss = self._run_step(batch)
        self.log("test_loss", loss, on_epoch=True, on_step=False, prog_bar=True)

In [46]:
module = RegressorModule(
    embed=module.embed,
    embed_size=emb_size,
    loss_fn=nn.L1Loss(),
    learning_rate=1e-12,
)

We create another `Trainer` object for our fine-tuning train. This time we include two callbacks, one for *early stopping*, and one for *model checkpointing*. The `patience` hyperparameter controls how many epochs we will wait for before stopping the training process if there is no improvement on the validation set. The best model based on validation set performance will be saved, and then reloaded for final evaluation on the test set.

In [47]:
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint

# Create callbacks
callbacks = [
    EarlyStopping(
        monitor="valid_loss",
        min_delta=0.00,
        patience=3,
        mode="min",
    ),
    ModelCheckpoint(
        filename="best",
        monitor="valid_loss",
        mode="min",
        save_top_k=1,
    ),
]

trainer = pl.Trainer(
    max_epochs=20,
    accelerator="auto",
    callbacks=callbacks,
    num_sanity_val_steps=0,  # Turn off sanity checking of validation dataloader
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [48]:
# Takes about 25 s per epoch
trainer.fit(
    model=module,
    train_dataloaders=train_dataloader,
    val_dataloaders=valid_dataloader
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params | Mode 
-----------------------------------------------
0 | embed   | EEGNetv4   | 152 K  | eval 
1 | clf     | Sequential | 101    | train
2 | loss_fn | L1Loss     | 0      | train
-----------------------------------------------
152 K     Trainable params
0         Non-trainable params
152 K     Total params
0.609     Total estimated model params size (MB)
4         Modules in train mode
24        Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

## Going further

This is it for this start kit! 🎉

To improve *performance*, next steps can include:
- Including more recordings into our SSL training dataset
- Improving the architecture (deeper/wider, handling channel positions, etc.)
- Explore different SSL pre-text trask
- Optimizing the hyperparameters (learning rate, batch size, etc.).

Have fun!